In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import re
from nltk.stem.porter import PorterStemmer

In [2]:
dataset=pd.read_csv("imdb_master.csv",encoding="ISO-8859-1")

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  100000 non-null  int64 
 1   type        100000 non-null  object
 2   review      100000 non-null  object
 3   label       100000 non-null  object
 4   file        100000 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [4]:
dataset=dataset.drop(["file","type","Unnamed: 0"],axis=1)


In [5]:
dataset.head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [6]:
test=dataset[dataset.label=="unsup"]

In [7]:
test.head()

,review,label
50000,"I admit, the great majority of films released ...",unsup
50001,"Take a low budget, inexperienced actors doubli...",unsup
50002,"Everybody has seen 'Back To The Future,' right...",unsup
50003,Doris Day was an icon of beauty in singing and...,unsup
50004,"After a series of silly, fun-loving movies, 19...",unsup


In [8]:
test=test.drop(["label"],axis=1)


In [9]:
test.head()

,review
50000,"I admit, the great majority of films released ..."
50001,"Take a low budget, inexperienced actors doubli..."
50002,"Everybody has seen 'Back To The Future,' right..."
50003,Doris Day was an icon of beauty in singing and...
50004,"After a series of silly, fun-loving movies, 19..."


In [10]:
dataset=dataset[dataset.label!="unsup"]

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  50000 non-null  object
 1   label   50000 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [12]:
dataset["label"]=dataset["label"].map({"pos":1,"neg":0})

In [13]:
dataset.head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [14]:
y=dataset["label"].values

In [15]:
y.shape=(-1,1)

In [16]:
dataset=dataset.drop(["label"],axis=1)

In [17]:
dataset.head()

,review
0,Once again Mr. Costner has dragged out a movie...
1,This is an example of why the majority of acti...
2,"First of all I hate those moronic rappers, who..."
3,Not even the Beatles could write songs everyon...
4,Brass pictures (movies is not a fitting word f...


In [20]:
from nlppreprocess import NLP
nlp=NLP()

In [56]:
test["review"]=test["review"].apply(nlp.process)

In [22]:
dataset["review"]=dataset["review"].apply(nlp.process)

In [24]:
dataset["review"]=dataset["review"].apply(lambda x:x.lower())

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=20000,oov_token="<oov>")

In [37]:
tokenizer.fit_on_texts(dataset["review"])

In [38]:
word_index=tokenizer.word_index

In [40]:
sequences=tokenizer.texts_to_sequences(dataset["review"])

In [45]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_seq=pad_sequences(sequences,maxlen=120)

In [57]:
test_sequence=tokenizer.texts_to_sequences(test["review"])

In [58]:
pad_test=pad_sequences(test_sequence,maxlen=120)

In [48]:
from tensorflow import keras
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Embedding
from keras.models import Sequential

In [49]:
model=Sequential()

In [51]:
model.add(Embedding(20000,16,input_length=120))

In [52]:
model.add(Flatten())
model.add(Dense(6,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [53]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [55]:
num_epochs=5
model.fit(pad_seq,y,epochs=num_epochs,validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 8s 199us/step - loss: 3.6513e-06 - accuracy: 1.0000 - val_loss: 4.0253e-06 - val_accuracy: 1.0000
Epoch 2/5
40000/40000 [==============================] - 8s 191us/step - loss: 1.7295e-06 - accuracy: 1.0000 - val_loss: 3.0144e-06 - val_accuracy: 1.0000
Epoch 3/5
40000/40000 [==============================] - 8s 200us/step - loss: 8.6041e-07 - accuracy: 1.0000 - val_loss: 2.4444e-06 - val_accuracy: 1.0000
Epoch 4/5
40000/40000 [==============================] - 8s 204us/step - loss: 4.4240e-07 - accuracy: 1.0000 - val_loss: 2.4175e-06 - val_accuracy: 1.0000
Epoch 5/5
40000/40000 [==============================] - 8s 206us/step - loss: 2.2949e-07 - accuracy: 1.0000 - val_loss: 1.9508e-06 - val_accuracy: 1.0000


In [59]:
test_pred=model.predict(pad_test)

In [61]:
test_pred=test_pred>0.5